In [1]:
import pandas as pd

In [28]:
from elasticsearch import Elasticsearch, helpers

In [2]:
df = pd.read_csv('../Data/data_2019_timestamp.csv')

In [3]:
df

Unnamed: 0 Park Name  Park DateTime Park Status  Occupated  Total  \
0                0      ANTI   1.546301e+09        Open         36    239   
1           251421      MOSS   1.546301e+09        Open         10    480   
2           291246      PITO   1.546301e+09        Open        102    588   
3           211416     GA250   1.546301e+09        Open         24    300   
4           311153      SABI   1.546301e+09        Open        138    485   
...            ...       ...            ...         ...        ...    ...   
409372      291245      OCCI   1.577836e+09        Open        159    700   
409373      370363      SJLC   1.577836e+09        Open          6    290   
409374       59328      CDGA   1.577836e+09        Open         32    260   
409375       78775      CIRC   1.577836e+09        Open        615   1500   
409376      409376      VICA   1.577836e+09        Open         10    150   

           x_pos      y_pos  
0       3.888819  43.608716  
1       3.819666  43.616237  
2       3.870191  43.612245  
3       3.914415  43.605344  
4       3.860225  43.583833  
...          ...        ...  
409372  3.848598  43.634562  
409373  3.837931  43.570822  
409374  3.897762  43.628542  
409375  3.917849  43.604954  
409376  3.898426  43.632771  

[409377 rows x 8 columns]

In [30]:
test = df.head(1)

In [8]:
test['Park DateTime'].astype('int32')

0     1546300800
1     1546300800
2     1546300800
3     1546300800
4     1546300800
         ...    
95    1546306800
96    1546306800
97    1546306800
98    1546306800
99    1546306800
Name: Park DateTime, Length: 100, dtype: int32

In [20]:
dic_to_es = {}

In [25]:
for index, row in test.iterrows():
    if row['Park DateTime'] in dic_to_es:
        temp_park_dic = {'Name' : row['Park Name'],'Status':row['Park Status'],'Free' : row['Total']-row['Occupated'], 
                         'Total': row['Total'], 'x_pos': row['x_pos'], 'y_pos':row['y_pos']}
        dic_to_es[row['Park DateTime']]['Parkings'].append(temp_park_dic)
    else:
        dic_to_es[row['Park DateTime']] = {'dateTime' : row['Park DateTime'], 'Parkings' : []}
        temp_park_dic = {'Name' : row['Park Name'],'Status':row['Park Status'],'Free' : row['Total']-row['Occupated'], 
                         'Total': row['Total'], 'x_pos': row['x_pos'], 'y_pos':row['y_pos']}
        dic_to_es[row['Park DateTime']]['Parkings'].append(temp_park_dic)
#     print(index)

In [26]:
len(dic_to_es)

25322

In [29]:
es = Elasticsearch(
    cloud_id='MarathonDuWeb:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJGZhMzJkNzYyNzFjYzQ4OTRhNjk2ZWQ4NTVhYjkwMDgyJGZjOTk1YzA0ZmYyYjQ5ZmM4OTliYjAwODNkYzA5N2U0',
    basic_auth=('elastic', 'QjTCAIOmAkkqqTIGjeEeD63q')
)

In [33]:
test_dic = dic_to_es[1546300800]

In [34]:
test_dic

{'dateTime': 1546300800.0,
 'Parkings': [{'Name': 'ANTI',
   'Status': 'Open',
   'Free': 203,
   'Total': 239,
   'x_pos': 3.888819,
   'y_pos': 43.608716},
  {'Name': 'MOSS',
   'Status': 'Open',
   'Free': 470,
   'Total': 480,
   'x_pos': 3.819666,
   'y_pos': 43.616237},
  {'Name': 'PITO',
   'Status': 'Open',
   'Free': 486,
   'Total': 588,
   'x_pos': 3.870191,
   'y_pos': 43.612245},
  {'Name': 'GA250',
   'Status': 'Open',
   'Free': 276,
   'Total': 300,
   'x_pos': 3.914415,
   'y_pos': 43.605344},
  {'Name': 'SABI',
   'Status': 'Open',
   'Free': 347,
   'Total': 485,
   'x_pos': 3.860225,
   'y_pos': 43.583833},
  {'Name': 'GARC',
   'Status': 'Open',
   'Free': 244,
   'Total': 395,
   'x_pos': 3.890716,
   'y_pos': 43.590985},
  {'Name': 'SABL',
   'Status': 'Open',
   'Free': 358,
   'Total': 372,
   'x_pos': 3.922295,
   'y_pos': 43.634192},
  {'Name': 'FOCH',
   'Status': 'Open',
   'Free': 181,
   'Total': 586,
   'x_pos': 3.876571,
   'y_pos': 43.610749},
  {'Name

In [35]:
es.index(
 index='es_test',
 document=test_dic)

ObjectApiResponse({'_index': 'es_test', '_id': 'AbJfaX8Bu-QUkx0oBeHH', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [36]:
result = es.search(
 index='es_test',
  query={
    'match': {'dateTime': 1546300800}
  }
 )

In [ ]:
es.indices.delete(index='test-index', ignore=[400, 404])

In [39]:
result['hits']['hits']

[{'_index': 'es_test',
  '_id': 'AbJfaX8Bu-QUkx0oBeHH',
  '_score': 1.0,
  '_source': {'dateTime': 1546300800.0,
   'Parkings': [{'Name': 'ANTI',
     'Status': 'Open',
     'Free': 203,
     'Total': 239,
     'x_pos': 3.888819,
     'y_pos': 43.608716},
    {'Name': 'MOSS',
     'Status': 'Open',
     'Free': 470,
     'Total': 480,
     'x_pos': 3.819666,
     'y_pos': 43.616237},
    {'Name': 'PITO',
     'Status': 'Open',
     'Free': 486,
     'Total': 588,
     'x_pos': 3.870191,
     'y_pos': 43.612245},
    {'Name': 'GA250',
     'Status': 'Open',
     'Free': 276,
     'Total': 300,
     'x_pos': 3.914415,
     'y_pos': 43.605344},
    {'Name': 'SABI',
     'Status': 'Open',
     'Free': 347,
     'Total': 485,
     'x_pos': 3.860225,
     'y_pos': 43.583833},
    {'Name': 'GARC',
     'Status': 'Open',
     'Free': 244,
     'Total': 395,
     'x_pos': 3.890716,
     'y_pos': 43.590985},
    {'Name': 'SABL',
     'Status': 'Open',
     'Free': 358,
     'Total': 372,
     'x